# Hypothesis Testing 

In this notebook we demonstrate formal hypothesis testing using the NHANES data.

It is important to note that the NHANES data are a "complex survey". The data are not an independent and representative sample from the target population. Proper analysis of complex survey data should make use of additional information about how the data were collected. Since complex survey analysis is a somewhat specialized topic, we ignore this aspect of the data here, and analyze the NHANES data as if it were an independent and identically distributed sample from a population.

First we import the libraries that we will need.

In [105]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

import statsmodels.api as sm
import scipy.stats.distributions as dist

Below we read the data, and convert some of the integer codes to text values. The NHANES codebooks for `SMQ020`, `RIAGENDR`, and `DMDCITZN` describe the meanings of the numerical codes.

In [3]:
da = pd.read_csv(r'C:\Users\Badr\Desktop\DA & DS in Python\Introduction to Hypothesis Testing in Python\nhanes_2015_2016.csv')
da.head(5)

,SEQN,ALQ101,ALQ110,ALQ130,SMQ020,RIAGENDR,RIDAGEYR,RIDRETH1,DMDCITZN,DMDEDUC2,...,BPXSY2,BPXDI2,BMXWT,BMXHT,BMXBMI,BMXLEG,BMXARML,BMXARMC,BMXWAIST,HIQ210
0,83732,1.0,NaN,1.0,1,1,62,3,1.0,5.0,...,124.0,64.0,94.8,184.5,27.8,43.3,43.6,35.9,101.1,2.0
1,83733,1.0,NaN,6.0,1,1,53,3,2.0,3.0,...,140.0,88.0,90.4,171.4,30.8,38.0,40.0,33.2,107.9,NaN
2,83734,1.0,NaN,NaN,1,1,78,3,1.0,3.0,...,132.0,44.0,83.4,170.1,28.8,35.6,37.0,31.0,116.5,2.0
3,83735,2.0,1.0,1.0,2,2,56,3,1.0,5.0,...,134.0,68.0,109.8,160.9,42.4,38.5,37.7,38.3,110.1,2.0
4,83736,2.0,1.0,1.0,2,2,42,4,1.0,4.0,...,114.0,54.0,55.2,164.9,20.3,37.4,36.0,27.2,80.4,2.0


In [9]:
da.columns

Index(['SEQN', 'ALQ101', 'ALQ110', 'ALQ130', 'SMQ020', 'RIAGENDR', 'RIDAGEYR',
       'RIDRETH1', 'DMDCITZN', 'DMDEDUC2', 'DMDMARTL', 'DMDHHSIZ', 'WTINT2YR',
       'SDMVPSU', 'SDMVSTRA', 'INDFMPIR', 'BPXSY1', 'BPXDI1', 'BPXSY2',
       'BPXDI2', 'BMXWT', 'BMXHT', 'BMXBMI', 'BMXLEG', 'BMXARML', 'BMXARMC',
       'BMXWAIST', 'HIQ210'],
      dtype='object')

In [4]:
da.SMQ020.unique()

array([1, 2, 7, 9], dtype=int64)

In [8]:
da['SMQ020'] = da['SMQ020'].replace({
    1 : "Yes",
    2 : "No",
    7 : np.nan,
    9 : np.nan
})
da.SMQ020.unique()

array(['Yes', 'No', nan], dtype=object)

In [10]:
da['RIAGENDR'].unique()

array([1, 2], dtype=int64)

In [12]:
da['RIAGENDR'] = da['RIAGENDR'].replace({
    1 : "Male",
    2 : "Female"
})
da['RIAGENDR'].unique()

array(['Male', 'Female'], dtype=object)

In [15]:
da['DMDCITZN'].unique()

array([ 1.,  2.,  9.,  7., nan])

In [18]:
da['DMDCITZN'] = da['DMDCITZN'].replace({
    1 : "Yes",
    2 : "No",
    7 : np.nan,
    9 : np.nan
})
da['DMDCITZN'].unique()

array(['Yes', 'No', nan], dtype=object)

In [19]:
da.head(5)

,SEQN,ALQ101,ALQ110,ALQ130,SMQ020,RIAGENDR,RIDAGEYR,RIDRETH1,DMDCITZN,DMDEDUC2,...,BPXSY2,BPXDI2,BMXWT,BMXHT,BMXBMI,BMXLEG,BMXARML,BMXARMC,BMXWAIST,HIQ210
0,83732,1.0,NaN,1.0,Yes,Male,62,3,Yes,5.0,...,124.0,64.0,94.8,184.5,27.8,43.3,43.6,35.9,101.1,2.0
1,83733,1.0,NaN,6.0,Yes,Male,53,3,No,3.0,...,140.0,88.0,90.4,171.4,30.8,38.0,40.0,33.2,107.9,NaN
2,83734,1.0,NaN,NaN,Yes,Male,78,3,Yes,3.0,...,132.0,44.0,83.4,170.1,28.8,35.6,37.0,31.0,116.5,2.0
3,83735,2.0,1.0,1.0,No,Female,56,3,Yes,5.0,...,134.0,68.0,109.8,160.9,42.4,38.5,37.7,38.3,110.1,2.0
4,83736,2.0,1.0,1.0,No,Female,42,4,Yes,4.0,...,114.0,54.0,55.2,164.9,20.3,37.4,36.0,27.2,80.4,2.0


##  1) Hypothesis Tests for one Proportion

The most basic hypothesis test may be the **one-sample test for a proportion**. This test is used if we have specified a particular value as the null value for the proportion, and we wish to assess if the data are compatible with the true parameter value being equal to this specified value. One-sample tests are not used very often in practice, because it is not very common that we have a specific fixed value to use for comparison.

For illustration, imagine that the rate of lifetime smoking in another country was known to be 40%, and we wished to assess whether the rate of lifetime smoking in the US were different from 40%. In the following notebook cell, we carry out the **(two-sided) one-sample test** that the population proportion of smokers is 0.4, and obtain a p-value of 0.43. This indicates that the NHANES data are compatible with the proportion of (ever) smokers in the US being 40%.

In [23]:
x = da['SMQ020'].dropna() == "Yes"

### <ins> **Method 1 :**</ins>

In [26]:
# Proportion of smokers (sample)
p = x.mean()
p

0.4050655021834061

In [27]:
#Standard Deviation
se = np.sqrt(0.4 * (1-0.4)/len(x))
se

0.00647467353462031

In [31]:
#Test statistic
test_stat = (p -.4) / se

#p-value (2sided):
pvalue = 2 * dist.norm.cdf(-np.abs(test_stat))

print("Test-statistic is : {}, and the p-value is : {}".format(test_stat, pvalue))

Test-statistic is : 0.7823563854332805, and the p-value is : 0.4340051581348052


### <ins> **Method 2 : statsmodels**</ins>

The following cell carries out the same test as performed above using the Statsmodels library. The results in the first (default) case below are slightly different from the results obtained above because Statsmodels by default uses the sample proportion instead of the null proportion when computing the standard error. This distinction is rarely consequential, but we can specify that the null proportion should be used to calculate the standard error, and the results agree exactly with what we calculated above. The first two lines below carry out tests using the normal approximation to the sampling distribution of the test statistic, and the third line below carries uses the exact binomial sampling distribution. We can see here that the p-values are nearly identical in all three cases. This is expected when the sample size is large, and the proportion is not close to either 0 or 1.



In [32]:
#Twoside is the default mode in sm.stats.proportions_ztest:
sm.stats.proportions_ztest(x.sum(), len(x), 0.4)

(0.7807518954896244, 0.43494843171868214)

In [34]:
# Binominal
p_value = sm.stats.binom_test(x.sum(), len(x), .4)
p_value

0.4340360854410028

## 2) Hypothesis Tests for two Proportions

Comparative tests tend to be used much more frequently than tests comparing one population to a fixed value. A **two-sample test of proportions** is used to assess whether the proportion of individuals with some trait differs between two sub-populations. For example, we can compare the smoking rates between females and males. Since smoking rates vary strongly with age, we do this in the subpopulation of people between 20 and 25 years of age. In the cell below, we carry out this test without using any libraries using Python code. We find that the smoking rate for men is around 10 percentage points greater than the smoking rate for females, and this difference is statistically significant (the p-value is around 0.01).

In [36]:
dx = da[["SMQ020", "RIDAGEYR", "RIAGENDR"]].dropna()
dx.head()

,SMQ020,RIDAGEYR,RIAGENDR
0,Yes,62,Male
1,Yes,53,Male
2,Yes,78,Male
3,No,56,Female
4,No,42,Female


In [47]:
dx.isna().sum()

SMQ020      0
RIDAGEYR    0
RIAGENDR    0
dtype: int64

### Method 1 :

In [48]:
#Proportion and number of smokers per gender:
p = dx.groupby('RIAGENDR')['SMQ020'].agg([lambda z : np.mean(z == "Yes"), "size"])
p.columns = ['Smoke', "N"]
print(p)

             Smoke     N
RIAGENDR                
Female    0.304845  2972
Male      0.513258  2753


Essentially the same test as above can be conducted by converting the "Yes"/"No" responses to numbers (Yes=1, No=0) and conducting a two-sample t-test, as below:

In [50]:
p_comb = (dx['SMQ020'] == "Yes").mean()
va = p_comb * (1 - p_comb)
se = np.sqrt(va * (1/p.N.Female + 1/p.N.Male))
(p_comb, va, se)

(0.4050655021834061, 0.2409874411243111, 0.01298546309757376)

In [51]:
test_stat = (p.Smoke.Female - p.Smoke.Male) / se
p_value = 2 * dist.norm.cdf(-np.abs(test_stat))
print("Test-statistic is : {}, and the p-value is : {}".format(test_stat, p_value))

Test-statistic is : -16.049719603652488, and the p-value is : 5.742288777302776e-58


### Method 2 (statsmodels) : 



In [52]:
dx['SMQ020'].unique()

array(['Yes', 'No'], dtype=object)

In [57]:
dx_females = dx.loc[dx['RIAGENDR'] == "Female", "SMQ020"].replace({
    "Yes" : 1,
    "No" : 0
})

In [55]:
dx_males = dx.loc[dx['RIAGENDR'] == "Male", "SMQ020"].replace({
    "Yes" : 1,
    "No" :0
})

In [59]:
sm.stats.ttest_ind(dx_females, dx_males)

(-16.42058555898443, 3.032088786691117e-59, 5723.0)

## 3) Hypothesis Tests Comparing Means

Tests of means are similar in many ways to tests of proportions. Just as with proportions, for comparing means there are **one** and **two-sample tests**, **z-tests** and **t-tests**, and **one-sided** and **two-sided tests**. As with tests of proportions, one-sample tests of means are not very common, but we illustrate a one sample test in the cell below. We compare systolic blood pressure `BPXSY1` to the fixed value 120 (which is the lower threshold for "pre-hypertension"), and find that the mean is significantly different from 120 (the point estimate of the mean is 126).

### One-Sample test

In [60]:
da.columns

Index(['SEQN', 'ALQ101', 'ALQ110', 'ALQ130', 'SMQ020', 'RIAGENDR', 'RIDAGEYR',
       'RIDRETH1', 'DMDCITZN', 'DMDEDUC2', 'DMDMARTL', 'DMDHHSIZ', 'WTINT2YR',
       'SDMVPSU', 'SDMVSTRA', 'INDFMPIR', 'BPXSY1', 'BPXDI1', 'BPXSY2',
       'BPXDI2', 'BMXWT', 'BMXHT', 'BMXBMI', 'BMXLEG', 'BMXARML', 'BMXARMC',
       'BMXWAIST', 'HIQ210'],
      dtype='object')

In [61]:
dx = da[['BPXSY1', 'RIDAGEYR', 'RIAGENDR']].dropna()
dx.head(5)

,BPXSY1,RIDAGEYR,RIAGENDR
0,128.0,62,Male
1,146.0,53,Male
2,138.0,78,Male
3,132.0,56,Female
4,100.0,42,Female


In [62]:
dx.isnull().sum()

BPXSY1      0
RIDAGEYR    0
RIAGENDR    0
dtype: int64

In [64]:
# Males with age between 40 and 50
dx = dx[(dx['RIDAGEYR']>=40) & (dx["RIDAGEYR"]<=50) & (dx['RIAGENDR'] == "Male")]
dx.shape

(421, 3)

In [66]:
# Average of blood pressure amongst this subset :
dx['BPXSY1'].mean()

125.86698337292161

In [69]:
test_stat, pvalue = sm.stats.ztest(dx['BPXSY1'], value = 120)
print("Test-statistic is : {}, and the p-value is : {}".format(test_stat, pvalue))

Test-statistic is : 7.469764137102597, and the p-value is : 8.033869113167905e-14


### Two-Sample Test

In the cell below, we carry out a formal test of the null hypothesis that the mean blood pressure for women between the ages of 50 and 60 is equal to the mean blood pressure of men between the ages of 50 and 60. The results indicate that while the mean systolic blood pressure for men is slightly greater than that for women (129 mm/Hg versus 128 mm/Hg), this difference is not statistically significant.

There are a number of different variants on the two-sample t-test. Two often-encountered variants are the t-test carried out using the t-distribution, and the t-test carried out using the normal approximation to the reference distribution of the test statistic, often called a z-test. Below we display results from both these testing approaches. When the sample size is large, the difference between the t-test and z-test is very small.

In [71]:
da.columns

Index(['SEQN', 'ALQ101', 'ALQ110', 'ALQ130', 'SMQ020', 'RIAGENDR', 'RIDAGEYR',
       'RIDRETH1', 'DMDCITZN', 'DMDEDUC2', 'DMDMARTL', 'DMDHHSIZ', 'WTINT2YR',
       'SDMVPSU', 'SDMVSTRA', 'INDFMPIR', 'BPXSY1', 'BPXDI1', 'BPXSY2',
       'BPXDI2', 'BMXWT', 'BMXHT', 'BMXBMI', 'BMXLEG', 'BMXARML', 'BMXARMC',
       'BMXWAIST', 'HIQ210'],
      dtype='object')

In [78]:
# People 50<= age <= 60 :
dx = da[['BPXSY1', 'RIDAGEYR', 'RIAGENDR']].dropna()
dx = dx.loc[(dx.RIDAGEYR >= 50) & (dx.RIDAGEYR <= 60), :]
dx.head()

,BPXSY1,RIDAGEYR,RIAGENDR
1,146.0,53,Male
3,132.0,56,Female
9,178.0,56,Male
15,134.0,57,Female
19,136.0,54,Female


In [86]:
# Division per gender:
bpx_female = dx.loc[dx.RIAGENDR == "Female", "BPXSY1"]
bpx_male = dx.loc[dx.RIAGENDR == "Male", "BPXSY1"]

(bpx_female.mean(), bpx_male.mean())


(127.92561983471074, 129.23829787234044)

In [88]:
# Z-test
sm.stats.ztest(bpx_female, bpx_male)

(-1.105435895556249, 0.2689707570859362)

In [89]:
# T-test
sm.stats.ttest_ind(bpx_female, bpx_male)

(-1.105435895556249, 0.26925004137768577, 952.0)

Another important aspect of two-sample mean testing is **"heteroscedasticity"**, meaning that the variances within the two groups being compared may be different. While the goal of the test is to compare the means, the variances play an important role in calibrating the statistics (deciding how big the mean difference needs to be to be declared statistically significant). In the NHANES data, we see that there are moderate differences between the amount of variation in BMI for females and for males, looking within 10-year age bands. In every age band, females having greater variation than males.

In [98]:
dx = da[['BPXSY1', 'RIDAGEYR', 'RIAGENDR']].dropna()
# age grouping
da['agegrp'] = pd.cut(da.RIDAGEYR, [18,30,40,50,60,70,80])
da.groupby(["agegrp", "RIAGENDR"])['BMXBMI'].agg(np.std).unstack()

RIAGENDR,Female,Male
agegrp,,
"(18, 30]",7.745893,6.649440
"(30, 40]",8.315608,6.622412
"(40, 50]",8.076195,6.407076
"(50, 60]",7.575848,5.914373
"(60, 70]",7.604514,5.933307
"(70, 80]",6.284968,4.974855


The standard error of the mean difference (e.g. mean female blood pressure minus mean male blood pressure) can be estimated in at least two different ways. In the statsmodels library, these approaches are referred to as the **"pooled"** and the **"unequal" approach to estimating the variance**. If the variances are equal (i.e. there is no heteroscedasticity), then there should be little difference between the two approaches. Even in the presence of moderate heteroscedasticity, as we have here, we can see that the results for the two methods are quite similar. Below we have a loop that considers each 10-year age band and assesses the evidence for a difference in mean BMI for women and for men. The results printed in each row of output are the test-statistic and p-value.µ

In [103]:
# k : keys for bins, v : values (dataframes)

for k, v in da.groupby("agegrp"):
    bmi_female = v.loc[v.RIAGENDR=="Female", "BMXBMI"].dropna()
    bmi_female = sm.stats.DescrStatsW(bmi_female)
    bmi_male = v.loc[v.RIAGENDR=="Male", "BMXBMI"].dropna()
    bmi_male = sm.stats.DescrStatsW(bmi_male)
    print(k)
    print("pooled: ", sm.stats.CompareMeans(bmi_female, bmi_male).ztest_ind(usevar='pooled'))
    print("unequal:", sm.stats.CompareMeans(bmi_female, bmi_male).ztest_ind(usevar='unequal'))
    print()

(18, 30]
pooled:  (1.7026932933643388, 0.08862548061449649)
unequal: (1.7174610823927268, 0.08589495934713022)

(30, 40]
pooled:  (1.4378280405644916, 0.1504828511464818)
unequal: (1.4437869620833494, 0.14879891057892475)

(40, 50]
pooled:  (2.8933761158070186, 0.003811246059501354)
unequal: (2.9678691663536725, 0.0029987194174035366)

(50, 60]
pooled:  (3.362108779981367, 0.0007734964571391746)
unequal: (3.375494390173923, 0.0007368319423226574)

(60, 70]
pooled:  (3.6172401442432753, 0.000297761021031936)
unequal: (3.62848309454456, 0.0002850914147149227)

(70, 80]
pooled:  (2.926729252512258, 0.0034254694144858636)
unequal: (2.937779886769224, 0.0033057163315192996)

